In [4]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import PromptTemplate, ChatPromptTemplate
from langchain_core.prompts import load_prompt
from langchain_core.runnables import RunnablePassthrough
from langchain_core.callbacks import StreamingStdOutCallbackHandler
from langchain_core.globals import set_llm_cache, set_debug
from langchain_core.caches import InMemoryCache
from langchain_community.cache import SQLiteCache

# set_llm_cache(InMemoryCache())
set_llm_cache(SQLiteCache("cache.db"))
set_debug(True)

chat = ChatOpenAI(
    temperature=0.1,
    streaming=True,
    callbacks=[StreamingStdOutCallbackHandler()]
)
chat.invoke("How to make italian pizza?")


[llm/start] [llm:ChatOpenAI] Entering LLM run with input:
{
  "prompts": [
    "Human: How to make italian pizza?"
  ]
}
To make Italian pizza, you will need the following ingredients:

- 2 1/4 teaspoons active dry yeast
- 1 1/2 cups warm water
- 3 1/2 cups all-purpose flour
- 2 teaspoons salt
- 2 tablespoons olive oil
- 1 can of crushed tomatoes
- 1 clove of garlic, minced
- 1 teaspoon dried oregano
- 1 teaspoon dried basil
- Salt and pepper to taste
- Fresh mozzarella cheese
- Fresh basil leaves

Here's how to make Italian pizza:

1. In a small bowl, dissolve the yeast in the warm water and let it sit for about 5 minutes until it becomes frothy.

2. In a large mixing bowl, combine the flour and salt. Make a well in the center and pour in the yeast mixture and olive oil. Stir until a dough forms.

3. Turn the dough out onto a floured surface and knead for about 5-7 minutes until it becomes smooth and elastic. Place the dough in a greased bowl, cover with a clean towel, and let it rise

AIMessage(content="To make Italian pizza, you will need the following ingredients:\n\n- 2 1/4 teaspoons active dry yeast\n- 1 1/2 cups warm water\n- 3 1/2 cups all-purpose flour\n- 2 teaspoons salt\n- 2 tablespoons olive oil\n- 1 can of crushed tomatoes\n- 1 clove of garlic, minced\n- 1 teaspoon dried oregano\n- 1 teaspoon dried basil\n- Salt and pepper to taste\n- Fresh mozzarella cheese\n- Fresh basil leaves\n\nHere's how to make Italian pizza:\n\n1. In a small bowl, dissolve the yeast in the warm water and let it sit for about 5 minutes until it becomes frothy.\n\n2. In a large mixing bowl, combine the flour and salt. Make a well in the center and pour in the yeast mixture and olive oil. Stir until a dough forms.\n\n3. Turn the dough out onto a floured surface and knead for about 5-7 minutes until it becomes smooth and elastic. Place the dough in a greased bowl, cover with a clean towel, and let it rise in a warm place for about 1-2 hours until it doubles in size.\n\n4. Preheat your

In [6]:
template = ChatPromptTemplate.from_messages([
    ("system", "You are a geography expert. And you only reply in {language}."),
    ("ai", "저는 {expert_class}입니다."),
    ("human", "{country_a}와 {country_b} 사이의 거리는 몇 km인가요? 당신의 역할도 알려주세요.")
])

prompt = template.format_messages(
    language="Korean",
    expert_class="지리 전문가",
    country_a="멕시코",
    country_b="태국"
)

chat.invoke(prompt)

AIMessage(content='멕시코와 태국 사이의 거리는 약 16,000km입니다. 제 역할은 지리 전문가입니다.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 38, 'prompt_tokens': 76, 'total_tokens': 114, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'id': 'chatcmpl-CaHx7ODHEcfdIoMFCazIKkGp2Vxt9', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='lc_run--3e08fd78-3c5a-454a-af00-789d5bb50843-0', usage_metadata={'input_tokens': 76, 'output_tokens': 38, 'total_tokens': 114, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

In [ ]:
from langchain_core.output_parsers import BaseOutputParser

class CommaOutputParser(BaseOutputParser):
    def parse(self, text):
        return [s for i in text.split(",") if (s := i.strip())]
    
parser = CommaOutputParser()
parser.parse("hello, how, are,    ")

chef_prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a world-class international chef. You create easy to follow recipies for any type of cuisine with easy to find ingredients."),
    ("human", "저는 {cuisine}을 요리하고 싶어요.")
])
chef_chain = chef_prompt | chat

veg_chef_prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a vegetarian chef specialized on making traditional recipies vegetarian. You find alternative ingredients and explain their preparation. You don't radically modify the recipe. If there is no alternative for a food just say you don't know how to replace it."),
    ("human", "{recipe}")
])
veg_chef_chain = veg_chef_prompt | chat

final_chain = {"recipe": chef_chain} | veg_chef_chain
final_chain.invoke({
    "cuisine": "칼국수"
})



칼국수는 한국의 전통 음식 중 하나로, 맑은 국물에 면발이 쫄깃한 요리입니다. 아래는 간단한 칼국수 레시피입니다.

재료:
- 칼국수 면 200g
- 물 6컵
- 다진 마늘 1큰술
- 다진 생갘 1작은술
- 소금 1작은술
- 참기름 1큰술
- 소고기 양지(또는 다른 고기) 100g
- 대파 1대
- 계란 1개
- 소금 약간
- 후추 약간

요리과정:
1. 냄비에 물을 붓고 소금을 넣어 끓입니다.
2. 끓는 물에 칼국수 면을 넣고 삶아줍니다. 삶는 동안 면이 서로 붙지 않도록 가끔씩 저어줍니다.
3. 면이 삶아지면 찬물에 헹궈 식힙니다.
4. 소고기 양지는 소금과 후추로 밑간을 해둡니다.
5. 팬에 식용유를 두르고 다진 마늘, 생강을 볶다가 소고기를 넣고 익힙니다.
6. 대파를 송송 썰어 준비합니다.
7. 냄비에 물 6컵을 붓고 소금을 넣어 끓입니다.
8. 끓는 물에 면을 넣고 삶아줍니다.
9. 면이 삶아지면 찬물에 헹궈 식힙니다.
10. 국그릇에 면을 담고 끓인 국물을 부어줍니다.
11. 소고기와 대파를 올려 마무리합니다.
12. 계란후라이를 곁들여 완성합니다.

이렇게 간단하게 맛있는 칼국수를 만들 수 있습니다. 맛있게 드세요!칼국수는 한국의 전통 음식 중 하나로, 면발이 쫄깃한 맑은 국물 요리입니다. 아래는 간단한 칼국수 레시피를 채식 버전으로 바꿔보겠습니다.

재료:
- 칼국수 면 200g
- 물 6컵
- 다진 마늘 1큰술
- 다진 생강 1작은술
- 소금 1작은술
- 참기름 1큰술
- 식물성 단백질 대체 고기 (예: 콩고기, 모밀고기) 100g
- 대파 1대
- 식물성 계란 대체품 1개
- 소금 약간
- 후추 약간

대체재료 및 요리과정:
1. 냄비에 물을 붓고 소금을 넣어 끓입니다.
2. 끓는 물에 칼국수 면을 넣고 삶아줍니다. 삶는 동안 면이 서로 붙지 않도록 가끔씩 저어줍니다.
3. 면이 삶아지면 찬물에 헹궈 식힙니다.
4. 식물성 단백질 대체 고기는 소금과 후추로 밑간을 해둡니다.
5. 팬에 식용유를 두르고 다진 마늘, 생강을 볶다가 대체 고기를 

AIMessage(content='칼국수는 한국의 전통 음식 중 하나로, 면발이 쫄깃한 맑은 국물 요리입니다. 아래는 간단한 칼국수 레시피를 채식 버전으로 바꿔보겠습니다.\n\n재료:\n- 칼국수 면 200g\n- 물 6컵\n- 다진 마늘 1큰술\n- 다진 생강 1작은술\n- 소금 1작은술\n- 참기름 1큰술\n- 식물성 단백질 대체 고기 (예: 콩고기, 모밀고기) 100g\n- 대파 1대\n- 식물성 계란 대체품 1개\n- 소금 약간\n- 후추 약간\n\n대체재료 및 요리과정:\n1. 냄비에 물을 붓고 소금을 넣어 끓입니다.\n2. 끓는 물에 칼국수 면을 넣고 삶아줍니다. 삶는 동안 면이 서로 붙지 않도록 가끔씩 저어줍니다.\n3. 면이 삶아지면 찬물에 헹궈 식힙니다.\n4. 식물성 단백질 대체 고기는 소금과 후추로 밑간을 해둡니다.\n5. 팬에 식용유를 두르고 다진 마늘, 생강을 볶다가 대체 고기를 넣고 익힙니다.\n6. 대파를 송송 썰어 준비합니다.\n7. 냄비에 물 6컵을 붓고 소금을 넣어 끓입니다.\n8. 끓는 물에 면을 넣고 삶아줍니다.\n9. 면이 삶아지면 찬물에 헹궈 식힙니다.\n10. 국그릇에 면을 담고 끓인 국물을 부어줍니다.\n11. 대체 고기와 대파를 올려 마무리합니다.\n12. 식물성 계란후라이를 곁들여 완성합니다.\n\n이렇게 대체재료를 활용하여 맛있는 채식 칼국수를 만들어보세요. 맛있게 드세요!', additional_kwargs={}, response_metadata={'finish_reason': 'stop', 'model_name': 'gpt-3.5-turbo-0125', 'service_tier': 'default', 'model_provider': 'openai'}, id='lc_run--783f40f7-0d57-4f89-8174-1afe5a9965df', usage_metadata={'input_tokens': 825, 'output_tokens': 682, 'total_tokens': 1507, 'input_token_d